In [1]:
# Read the CSV file
root_path = '/home/iceberg/data'

input_path = f'{root_path}/gem/appsflyer/installs_report/app_id={{id123456789,com.appsflyer.referrersender}}/event_date=2024-05-20'

df = spark.read.format('csv').options(header='true').load(input_path)

df = spark.sql("""
select event_name, event_time, install_time, af_prt, media_source, campaign, af_c_id, country_code, appsflyer_id, platform
from {installs_report}
where af_c_id is not null
""", installs_report = df)


# Show the data
df.show()

26/01/15 04:22:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------------------+-------------------+------+----------------+--------------------+----------+------------+--------------------+--------+
|event_name|         event_time|       install_time|af_prt|    media_source|            campaign|   af_c_id|country_code|        appsflyer_id|platform|
+----------+-------------------+-------------------+------+----------------+--------------------+----------+------------+--------------------+--------+
|   install|2024-05-20 23:44:46|2024-05-20 23:44:46|  NULL|Apple Search Ads|912_VHDL_FA Plan_...|1576814064|          VN|1716213676770-618...|     ios|
|   install|2024-05-20 23:42:37|2024-05-20 23:42:37|  NULL|Apple Search Ads|912_VHDL_SearchRe...|1575425784|          VN|1716213995151-399...|     ios|
|   install|2024-05-20 23:39:14|2024-05-20 23:39:14|  NULL|Apple Search Ads|912_VHDL_SearchRe...|1575425784|          VN|1716213809763-606...|     ios|
|   install|2024-05-20 23:34:37|2024-05-20 23:34:37|  NULL|Apple Search Ads|912_VHDL_Sea

In [7]:
root_path = '/home/iceberg/warehouse'
event_date = '2024-05-21'

df = spark.read.parquet(f'{root_path}/gem/reporting/in_app_events/event_date={event_date}')

df.select('event_name').distinct().show(100)

+--------------------+
|          event_name|
+--------------------+
|af_first_login_su...|
|          af_level_7|
|   af_Get_Pay_Vip 10|
|    af_Get_Pay_Vip 2|
|      af_first_gacha|
| af_press_start_game|
|   af_resource_start|
|         af_level_20|
|SDK_API_DETECT_CO...|
|    af_Get_Pay_Vip 9|
|af_enter_update_view|
|         af_level_24|
|   SDK_DIFF_LANGUAGE|
|        Installation|
|  af_resource_finish|
|af_first_role_ren...|
|          af_level_4|
|  af_server_selected|
|SDK_API_GET_BOOTS...|
|         af_purchase|
|af_first_pay_pack...|
|      af_role_create|
|         af_level_10|
|af_first_role_create|
|        registration|
|   af_tutorial_start|
|SDK_CLICK_LOGIN_L...|
|    af_Get_Pay_Vip 1|
|    af_login_success|
|  af_end_update_view|
|af_first_pay_pack...|
|      SDK_LOGIN_FLOW|
|     af_tutorial_end|
|        installation|
+--------------------+



In [3]:
from pyspark.sql.functions import *

root_path = '/home/iceberg/warehouse'
app_id = '{id123456789,com.appsflyer.referrersender}'
event_date = '2024-05-21'

input_path = f'{root_path}/gem/appsflyer/in_app_events_report/app_id={app_id}/event_date={event_date}'

df = spark.read.format('csv').options(header='true').load(input_path)

df = df.where("event_name like 'af_level_%'")

json_schema = spark.read.json(df.rdd.map(lambda row: row.event_value)).schema

df = df.withColumn('event_value', from_json(col('event_value'), json_schema))

df.printSchema()

df.show(truncate=False)

root
 |-- attributed_touch_type: string (nullable = true)
 |-- attributed_touch_time: string (nullable = true)
 |-- install_time: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_value: struct (nullable = true)
 |    |-- _corrupt_record: string (nullable = true)
 |-- event_revenue: string (nullable = true)
 |-- event_revenue_currency: string (nullable = true)
 |-- event_revenue_usd: string (nullable = true)
 |-- event_source: string (nullable = true)
 |-- is_receipt_validated: string (nullable = true)
 |-- af_prt: string (nullable = true)
 |-- media_source: string (nullable = true)
 |-- af_channel: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- af_c_id: string (nullable = true)
 |-- af_adset: string (nullable = true)
 |-- af_adset_id: string (nullable = true)
 |-- af_ad: string (nullable = true)
 |-- af_ad_id: string (nullable = true)
 |-- af_ad_type: st

26/01/12 10:52:11 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 374189 ms exceeds timeout 120000 ms
26/01/12 10:52:11 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/12 10:52:13 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [8]:
root_path = '/home/iceberg/warehouse'
event_date = '2024-05-21'

df = spark.read.parquet(f'{root_path}/gem/users/purchase/event_date={event_date}')

df.show()

+----------+-------------------+----------------------+-------------------+-------------------+---------+------------------+------------------+
|product_id|            user_id|event_revenue_currency| first_payment_time|  last_payment_time|  revenue|       revenue_usd|number_of_payments|
+----------+-------------------+----------------------+-------------------+-------------------+---------+------------------+------------------+
|       gem|1176732967775498240|                   VND|2024-05-21 10:20:19|2024-05-21 11:02:52|1556000.0| 61.12384712593925|                 6|
|       gem|1193843416465596416|                   VND|2024-05-21 10:39:54|2024-05-21 10:39:54|  10000.0|0.3928269793569422|                 1|
|       gem|1199550335929909248|                   VND|2024-05-21 11:03:30|2024-05-21 11:09:26| 2.2824E7| 896.5855687118491|                48|
|       gem|1232240346301456384|                   VND|2024-05-21 16:15:05|2024-05-21 16:23:46|  45000.0|1.7673395648417247|            

In [3]:
root_path = '/home/iceberg/warehouse'
event_date = '2024-05-21'

df = spark.read.parquet(f'{root_path}/gem/user_profile')

df.show()

+----------+-------------------+-------------------+-------------------+------+------------+-----------+------------+--------+--------------------+-------------------+-------------------+-------------+-------------------+-------------------+------------------+------------------+
|product_id|            user_id|       install_time|      register_time|agency|media_source|campaign_id|country_code|platform|          install_id|   first_login_time|    last_login_time|session_count| first_payment_time|  last_payment_time|       revenue_usd|number_of_payments|
+----------+-------------------+-------------------+-------------------+------+------------+-----------+------------+--------+--------------------+-------------------+-------------------+-------------+-------------------+-------------------+------------------+------------------+
|       gem|1174568175868424192|               NULL|               NULL|  NULL|        NULL|       NULL|        NULL|    NULL|                NULL|2024-05-21 10

In [1]:
df = spark.read.parquet("s3a://lakehouse/gem/raw/appsflyer/event_name=install/event_date=2024-05-20")
df.show(1)

26/01/15 10:21:35 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
26/01/15 10:21:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------------+---------------------+-------------------+-------------------+------+------------+----------+--------+--------+-------+--------+-----------+-----+--------+----------+----------+-------------+-------------+----------------+-------------------+-------------------------+---------------------+-----------------------+-----------------------+-------------------+-------------------------+---------------------+-----------------------+-----------------------+-------------------+-------------------------+---------------------+-----------------------+-----------------------+------+------------+-----+--------+-----------+------------+-----+--------+-------+--------+--------------------+--------------+---------+----------+----------------+----+--------------------+--------+-----------+----------+-----------+-----------+--------------------+--------------------+--------------+---------------------------+--------------------+----------------------+----------------------+-------

In [ ]:
df = spark.read.parquet("s3a://lakehouse/gem/curated/daily/installs/event_date=2024-05-20")
df.show(1)

26/01/16 04:01:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+----------+------+---------------+-----------+------------+--------+--------+
|product_id|agency|   media_source|campaign_id|country_code|platform|installs|
+----------+------+---------------+-----------+------------+--------+--------+
|       gem|    na|Social_facebook|         na|          VN|     ios|      43|
+----------+------+---------------+-----------+------------+--------+--------+
only showing top 1 row

